# ### Anaconda & Python Version:

The version of the notebook server is 5.7.0 and is running on:
Python 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]

Current Kernel Information:
Python 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
Type 'copyright', 'credits' or 'license' for more information
IPython 7.0.1 -- An enhanced Interactive Python. Type '?' for help.

#### Anaconda 5.0.1

In [1]:
# Versions of Libraries: (Code: 'pip list' in anaconda prompt)
# Check version of anaconda - 'conda list anaconda$' or 'conda info' in anaconda command prompt
# Check version of python - 'python' in anaconda command prompt

In [2]:
### Problem: Recommendation System for 1/2/3/4/5 ratings recommendations
### Problem Type: 
### Steps:
#### 0. 
#### 1. 
#### 2. Note: Should have a validation set also
#### 3. 
#### 4. 
#### 5. 
#### 6. 
#### 7.

In [3]:
## install pytorch:
# pytorch - (wrapper library over torch) is easiest and fastest for Boltzmann Machines, compared to tensorflow, keras
# pytorch - developed by Facebook AI research group, good for NLP, DL on GPUs
# keras - (wrapper library over tensorflow/theano), makes DL more intuitive and easy to use but no enhancement in speed

# installing pytorch:
# go to: https://pytorch.org/; select Stable, Windows, Pip, 3.6, None;
# change 'pip3' to 'pip' in below link:
# run "pip install http://download.pytorch.org/whl/cpu/torch-0.4.1-cp36-cp36m-win_amd64.whl" in anaconda cmd
# succefully installs: torch-0.4.1            # although author installed 'torch 0.1.10.post1'

In [4]:
## standard datasets for movie recommendation system:
# link - https://grouplens.org/datasets/movielens/
# for this problem, download 'ml-100k.zip' and 'ml-1m.zip' (same on grouplens and superdatascience)

In [5]:
## Special Note: Here we are going to work on torch tensors and not tensorflow tensors

In [12]:
try:
    import torch
except ImportError as e:
    !pip install torch

In [13]:
## import libraries:
import numpy as np
import pandas as pd
# import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [15]:
## import dataset:
movies = pd.read_csv('/home/antrived/Dropbox/Job-25-Feb-2019/prep/13. Data/1. Real Data/12. Recommendatio Engine Data/Movielens/ml-1m/movies.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
# movie names have ',', separator is '::', movie names may have special character so encoder 'utf-8' not used
# movies.head(3) # 4000 movies
movies=movies.rename(columns = {0:'Movie_ID'})
movies=movies.rename(columns = {1:'Movie_Title'})
movies=movies.rename(columns = {2:'Movie_Genre'})
movies.head(3)

,Movie_ID,Movie_Title,Movie_Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [16]:
## import dataset:
users = pd.read_csv('/home/antrived/Dropbox/Job-25-Feb-2019/prep/13. Data/1. Real Data/12. Recommendatio Engine Data/Movielens/ml-1m/users.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
users.head(3) # 6000 users; ID, Gender, Age, Job Code, Zip Code
users=users.rename(columns = {0:'User_ID'})
users=users.rename(columns = {1:'Gender'})
users=users.rename(columns = {2:'Age'})
users=users.rename(columns = {3:'Job_Code'})
users=users.rename(columns = {4:'Zip_Code'})
users.head(3)

,User_ID,Gender,Age,Job_Code,Zip_Code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [17]:
## import dataset:
ratings = pd.read_csv('/home/antrived/Dropbox/Job-25-Feb-2019/prep/13. Data/1. Real Data/12. Recommendatio Engine Data/Movielens/ml-1m/ratings.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
# ratings.head(3) # 1,000,000 ratings; User, Movie, Rating, timestamp when user rated movie
ratings=ratings.rename(columns = {0:'User_ID'})
ratings=ratings.rename(columns = {1:'Movie_ID'})
ratings=ratings.rename(columns = {2:'Rating'})
ratings=ratings.rename(columns = {3:'Time_Stamp'})
ratings.head(3)

,User_ID,Movie_ID,Rating,Time_Stamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


#######################################################################################
############################### Data Pre-Processing ###################################
#######################################################################################

In [11]:
## reduce Train-Univ size (for faster execution)
# remove users
ratings_small = ratings.head(300000)
ratings_small # 1779 users, ~4000 movies
ratings = ratings_small

In [12]:
## Normal train test split
# 'same user same set' will not work for recomm engine because a user should have watched some movies for recomm to generate

# random sampling
np.random.seed(4)
msk = np.random.rand(len(ratings)) < 0.75                        
Train = ratings[msk] # generates random sample of nearly 75%        
Test = ratings[~msk]

# drop Time_Stamp as not required
Train = Train.drop(['Time_Stamp'], axis=1)
Test = Test.drop(['Time_Stamp'], axis=1)

# save Train & Test as csv
Train.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Train.csv', sep=',')
Test.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Test.csv', sep=',')

In [13]:
## converting train-test to np array
Train_arr = np.array(Train, dtype='int')
Test_arr = np.array(Test, dtype='int')

In [14]:
## creating Train & Test user-movie 2d array - list of lists, a row represents list of a user with all movie ratings
# list of lists is required to create train/test torch tensors for NN
# each matrix will have all train-test users and movies as rows and columns respectively
# If user did not rate a movie, 0 will be put there

# getting the number of user and movies
frames = [Train, Test]
Train_Test_concat = pd.concat(frames) # 300,000 rows 
nb_users = Train_Test_concat.iloc[:,0].nunique() # total no. of users = 1779
nb_movies = Train_Test_concat.iloc[:,1].nunique() # total no. of movies = 3569

# function to create list of lists for train/test - same structure of all_users*all_movies

Train_Test_concat = Train_Test_concat.sort_values(['User_ID','Movie_ID'], ascending=[True,True])

def convert(Data):
    # create dataframe of user-movie(row*col) -> convert to list of list -> convert to torch tensor
    Data_list = Train_Test_concat[['User_ID', 'Movie_ID']]
    Data_list = pd.merge(Data_list, Data, how='left', on=['User_ID', 'Movie_ID'])
    # Train_list/Test _list # 300,000 rows
    
    Data_list['Rating'].fillna(0, inplace=True) # replace NA with 0 - did not watch
    # Train_list/Test_list # ratings will be correct only w.r.t. train/test set
    # Transpose
    Data_list = Data_list.pivot_table(index=['User_ID'],columns=['Movie_ID'],values='Rating',aggfunc='first',fill_value=0)
    
    # creating list of lists
    Temp = []
    for i in range(len(Data_list.index)):
        Temp1 = []
        for j in range(len(Data_list.columns)):
            Temp1.append(Data_list.iloc[i,j])
        Temp.append(Temp1)
    return Temp

Train_list = convert(Train) 
Test_list = convert(Test) 
# Temp # list of list is created with 1779 lists and 3569 values in each list

# check the structure using pd df
# Temp2 = pd.DataFrame(Train_list)
# Temp2 # 1779 × 3569
# Temp2 = pd.DataFrame(Test_list)
# Temp2 # 1779 × 3569

In [16]:
## converting data into torch tensors
# torch tensors are basically multi-dimensional arrays with same datatype, but compatible to torch
# author says NN with torch tensors performed better than NN with tensorflow  tensors (for BM and Autoencoders)
Train_torchtensor = torch.FloatTensor(Train_list)
Test_torchtensor = torch.FloatTensor(Test_list)

In [17]:
## Note: Here we are keeping 1-5 rating; 0 means user has not rated

#######################################################################################
################## Creating architecture of Stacked Autoencoders ######################
#######################################################################################

In [19]:
## creating architecture of Stacked Autoencoders - in pytorch
# will create inheritance class to use funcs & vars of parent class 'Module' in torch.nn lib; 
# bcoz parent class has all tools to make Autoencoder

nb_neurons_h1 = 20 # no of neurons in hidden layer 1; to be tuned
nb_neurons_h2 = 10 # no of neurons in hidden layer 2; to be tuned
nb_neurons_h3 = 20 # no of neurons in hidden layer 3; to be tuned
                   # last layer has been increased to 20 again as decoding would start from here
                   # 'number of layers' also to be tuned
                   # note: nb_movies/input neurons = 3569

class SAE(nn.Module): # creating inherited class from 'Module' class in nn lib/module
    def __init__(self, ): # to intialize the object
        super(SAE, self).__init__() # process of inheritance is done but we can optimize it using 'super' fn
                                    # using super function to inherit all fn/vars from 'nn.Module' class
        self.fc1 = nn.Linear(nb_movies, nb_neurons_h1) # fc=full_conn; (par1,par2) are no of input,hidden neurons for layer 1
        self.fc2 = nn.Linear(nb_neurons_h1, nb_neurons_h2) # (par1,par2) are no of input,hidden neurons for layer 2
        self.fc3 = nn.Linear(nb_neurons_h2, nb_neurons_h3)
        self.fc4 = nn.Linear(nb_neurons_h3, nb_movies)
        # note: neurons in hidden layers represent some characteristics of movie like neuron1=horror_genre and so on
        self.activation = nn.Sigmoid() # author said Sigmoid performed better than Rectifier # to be tuned
    def forward(self, x): # encoding and decoding function # x is input vector for a hidden layer
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x # vector of predicted ratings

In [20]:
## creating AE object/model; setting parameters

sae = SAE() # parameters are already defined in the class

criterion = nn.MSELoss()

lr_val=0.01 # lr: to be tuned
weight_decay_val = 0.5 # weight_decay: to be tuned
# weight decay: weights are reduced by a proportion to control overfitting
    # https://stats.stackexchange.com/questions/29130/difference-between-neural-net-weight-decay-and-learning-rate
optimizer = optim.RMSprop(sae.parameters(), lr=lr_val, weight_decay=weight_decay_val) 
# nb_neurons_h1, nb_neurons_h2 and so on are also tuning paramaters - mentioned above
# there is class for 'adam', 'rmsprop' etc, author got good result with rmsprop # to be tuned

In [21]:
# training the SAE

# weights to update after each iteration

nb_epochs = 200 # to be tuned
for epoch in range(1, nb_epochs+1):
    train_loss = 0
    s = 0. # initalized as float
    for id_user in range(nb_users):
        input = Variable(Train_torchtensor[id_user]).unsqueeze(0) # pytorch takes only 2D vectors # this improvization done to input 1D as 2D
        target = input.clone() # target same as input
        if torch.sum(target.data > 0) > 0: # train only on users rated at least 1 movie
            output = sae(input) # calls forward function in SAE class
            target.require_grad = False # code optim - calculates gradient only for input and not target
            output[target == 0] = 0     # code optim - absent ratings will not be counted in error calculation/weight update
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # mean adj fact; smallval added to avoid 0 in denom 
            loss.backward() # decides whether weights would be inc/dec in backpropagation
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step() # decided amount of weight change
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

# here error is RMSE
# Meaning - If error is 0.9 it means average difference between each pred and actual rating is 0.9 stars
# here targeting a loss of less than 1 star
# these type of problems should have vald set, so creating it now
# 0.8984 error with 75-25 sampling

C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch: 1 loss: tensor(1.8712)
epoch: 2 loss: tensor(1.1293)
epoch: 3 loss: tensor(1.0465)
epoch: 4 loss: tensor(1.0169)
epoch: 5 loss: tensor(1.0026)
epoch: 6 loss: tensor(0.9938)
epoch: 7 loss: tensor(0.9887)
epoch: 8 loss: tensor(0.9849)
epoch: 9 loss: tensor(0.9828)
epoch: 10 loss: tensor(0.9807)
epoch: 11 loss: tensor(0.9791)
epoch: 12 loss: tensor(0.9781)
epoch: 13 loss: tensor(0.9773)
epoch: 14 loss: tensor(0.9764)
epoch: 15 loss: tensor(0.9760)
epoch: 16 loss: tensor(0.9754)
epoch: 17 loss: tensor(0.9750)
epoch: 18 loss: tensor(0.9746)
epoch: 19 loss: tensor(0.9741)
epoch: 20 loss: tensor(0.9738)
epoch: 21 loss: tensor(0.9739)
epoch: 22 loss: tensor(0.9737)
epoch: 23 loss: tensor(0.9732)
epoch: 24 loss: tensor(0.9734)
epoch: 25 loss: tensor(0.9730)
epoch: 26 loss: tensor(0.9730)
epoch: 27 loss: tensor(0.9726)
epoch: 28 loss: tensor(0.9729)
epoch: 29 loss: tensor(0.9728)
epoch: 30 loss: tensor(0.9726)
epoch: 31 loss: tensor(0.9728)
epoch: 32 loss: tensor(0.9726)
epoch: 33 loss: t

In [22]:
## predicting on test data - calculating accuracy / test_loss

test_loss = 0
s = 0. # initalized as float
for id_user in range(nb_users):
    input = Variable(Train_torchtensor[id_user]).unsqueeze(0) # pytorch takes only 2D vectors # this improvization done to input 1D as 2D
    target = Variable(Test_torchtensor[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0: # train only on users rated at least 1 movie
        output = sae(input) # calls forward function in SAE class
        target.require_grad = False # code optim - calculates gradient only for input and not target
        output[target == 0] = 0     # code optim - absent ratings will not be counted in error calculation/weight update
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # mean adj fact; smallval added to avoid 0 in denom 
        # loss.backward() # decides whether weights would be inc/dec in backpropagation
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
        # optimizer.step() # decided amount of weight change
print('test_loss: '+str(test_loss/s))

# test loss = 0.9195

C:\Users\anurag.trivedi\Anaconda1\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


test_loss: tensor(0.9184)


In [23]:
## predicting on test data - calculating accuracy / test_loss
## & creating an output with all recommendations

# this code needs to be written - will be similar to previous para

output[0] # shows how output looks

tensor([4.0532, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
       grad_fn=<SelectBackward>)

In [ ]:
## also above code should be re-written with 2 vald sets, and auto parameter tuning

In [ ]:
## QC: this paragraph can be deleted:
# checking how predictions vs actuals are for last user

# actual of last user
Temp = pd.DataFrame(Test_list)
Temp.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Temp.csv', sep=',')
# focus on last user (actual)

# prediction of last user
temp1 = output.detach().numpy()
temp1 = pd.DataFrame(temp1[0]) # to convert 2D to 1D & array to df
temp1.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Temp1.csv', sep=',')

# Note: Predictions are generated for all non-zero rating in test, all predictions are between 1 to 5
# By manual check its clear actual and preds are generally within 1 rating range; which is good

In [32]:
## Error Metric and Activation Function:
# RMSE used here because of 1-5 predictions; Confusion Matrix can also be used
# MAE may not be the best to optimize; can optimize using RMSE but report MAE
## Activation Fn - Sigmoid (same for RBM)
# Why Sigmoid - Maybe because only 1 hidden layer iteration(forward pass) before weight updation, 
#               so Relu's benefits of faster weight update and lesser risk of Van. Gradient do not apply
#               Hence a more complex(non-linear) activation function used

#######################################################################################
################## Stacked Autoencoders - Scoring on new data ######################
#######################################################################################

In [ ]:
# 1. Try to find existing packages
# 2. Custom approach:
    # find existing user closest to new user using RMSE(not cosine sim here)
    # use this existing user's recomm for the new user